<a href="https://colab.research.google.com/github/Saipavani3891/FMML_PROJECTS_AND_LABS/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [1]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [2]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

:Number of Instances: 20640

:Number of Attributes: 8 numeric, predictive attributes and the target

:Attribute Information:
    - MedInc        median income in block group
    - HouseAge      median house age in block group
    - AveRooms      average number of rooms per household
    - AveBedrms     average number of bedrooms per household
    - Population    block group population
    - AveOccup      average number of household members
    - Latitude      block group latitude
    - Longitude     block group longitude

:Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived from the 1990 U.S. census, using one row per ce

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [3]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [4]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [5]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [6]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [7]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [8]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [9]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [10]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [11]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [12]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [13]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [14]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [15]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.

Yes, averaging the validation accuracy across multiple splits can provide more consistent results. This approach is known as k-fold cross-validation.

Here's why:

Reducing Variance

1. Single split bias: When you split your data into training and validation sets only once, you may introduce bias. The chosen split might not be representative of the overall data distribution.
2. Averaging reduces variance: By averaging validation accuracy across multiple splits, you reduce the variance caused by individual split biases.

Benefits

1. Improved estimate of model performance: Averaging across multiple splits provides a more reliable estimate of the model's performance on unseen data.
2. Increased robustness: This approach helps mitigate the impact of outliers or noisy data in individual splits.
3. Better hyperparameter tuning: When using k-fold cross-validation for hyperparameter tuning, averaging validation accuracy helps identify the most robust hyperparameters.

Considerations

1. Computational cost: Performing k-fold cross-validation can be computationally expensive, especially for large datasets or complex models.
2. Number of folds (k): Choosing the right value for k is crucial. Common values are 5 or 10. Higher values provide more robust estimates but increase computational cost.
3. Data diversity: If the data is highly diverse or has many outliers, more folds might be necessary to achieve consistent results.

To implement k-fold cross-validation, you can use popular libraries like scikit-learn (Python) or caret (R). These libraries provide efficient and easy-to-use functions for k-fold cross-validation.

Here's a simple example using scikit-learn in Python:



from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

# Define the model and data
X, y = ...  # Your data
model = LogisticRegression()

# Define k-fold cross-validation
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# Initialize list to store validation accuracies
val_accuracies = []

# Perform k-fold cross-validation
for train_index, val_index in kf.split(X):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    # Train the model
    model.fit(X_train, y_train)

    # Predict and calculate validation accuracy
    y_pred = model.predict(X_val)
    val_accuracy = accuracy_score(y_val, y_pred)

    # Store validation accuracy
    val_accuracies.append(val_accuracy)

# Calculate average validation accuracy
avg_val_accuracy = sum(val_accuracies) / k
print("Average validation accuracy:", avg_val_accuracy)


By averaging validation accuracy across multiple splits, you can increase the reliability of your results and gain a better understanding of your model's performance.

Averaging validation accuracy across multiple splits (k-fold cross-validation) provides a more accurate estimate of test accuracy under certain conditions.

Why it helps:

1. Reduces overfitting bias: When using a single validation set, the model may overfit to that specific set. K-fold cross-validation reduces this bias by evaluating the model on multiple, independent validation sets.
2. Better represents data distribution: Averaging across multiple splits helps to account for variations in the data distribution, providing a more robust estimate of test accuracy.
3. Mitigates variability: K-fold cross-validation reduces the impact of random fluctuations in validation accuracy, providing a more stable estimate.

When it's most effective:

1. Small to moderate-sized datasets: K-fold cross-validation is particularly useful when working with smaller datasets, where a single validation set might not be representative.
2. High variance models: Models with high variance (e.g., complex neural networks) benefit from k-fold cross-validation, as it helps to reduce overfitting.
3. Limited test data: When test data is scarce, k-fold cross-validation can provide a more accurate estimate of test accuracy.

Limitations:

1. Computational cost: Performing k-fold cross-validation can be computationally expensive, especially for large datasets or complex models.
2. Not a guarantee: While k-fold cross-validation can provide a more accurate estimate, it's not a guarantee. The test accuracy may still differ due to various factors (e.g., data drift, sampling bias).
3. Optimization bias: If hyperparameters are optimized using k-fold cross-validation, there's a risk of optimization bias, where the model is overfitted to the validation sets.

Best practices:

1. Use stratified k-fold cross-validation for classification problems to maintain class balance.
2. Choose an appropriate k (e.g., 5, 10) considering computational cost and data size.
3. Monitor validation accuracy variability to ensure k-fold cross-validation is effective.

By using k-fold cross-validation, you can increase the reliability of your validation accuracy estimate and gain a better understanding of your model's performance on unseen data.

he number of iterations (k) in k-fold cross-validation affects the estimate of test accuracy. Generally, increasing k provides a better estimate, but with diminishing returns and increased computational cost.

Effects of increasing k:

Positive effects:

1. Reduced variance: More iterations reduce the variance of the estimate, providing a more stable and reliable measure of test accuracy.
2. Improved bias-variance tradeoff: Increasing k helps to reduce overfitting bias and underfitting variance.
3. Better representation of data distribution: More iterations capture more aspects of the data distribution, leading to a more accurate estimate.

Negative effects:

1. Increased computational cost: More iterations require more computations, which can be time-consuming and resource-intensive.
2. Overfitting to validation sets: If k is too high, the model may overfit to the validation sets, leading to optimistic bias.
3. Optimization bias: Excessive iterations can result in optimization bias when tuning hyperparameters.

Optimal k value:

The optimal k value depends on:

1. Dataset size: Larger datasets require fewer iterations (e.g., k=5).
2. Model complexity: Simpler models require fewer iterations (e.g., k=5).
3. Computational resources: Balance computational cost with desired accuracy.

Common k values:

- k=5: A popular choice for many applications.
- k=10: Provides a better estimate, but increases computational cost.
- k=20 or more: Typically used for small datasets or complex models.

Asymptotic behavior:

As k approaches the dataset size (n), the estimate converges to the leave-one-out cross-validation (LOOCV) estimate. However, LOOCV is often computationally prohibitive.

Guidelines:

1. Start with k=5 and adjust based on dataset size and computational resources.
2. Monitor convergence: If the estimate stabilizes with increasing k, further iterations may not provide significant benefits.
3. Consider stratified k-fold cross-validation for classification problems.

By choosing an appropriate k value, you can balance computational cost and accuracy to obtain a reliable estimate of test accuracy.

Would you like me to elaborate on any of these points or provide examples?

Increasing iterations (k) in k-fold cross-validation can help alleviate issues with small train or validation datasets, but there are limitations.

Benefits for small datasets:

1. Reduced variance: More iterations help reduce the variance of the estimate, providing a more stable measure of test accuracy.
2. Better representation of data distribution: Increased iterations capture more aspects of the data distribution.
3. Improved bias-variance tradeoff: More iterations help reduce overfitting bias and underfitting variance.

Limitations:

1. Computational cost: Increasing iterations increases computational cost, potentially becoming prohibitive.
2. Optimization bias: Excessive iterations can lead to optimization bias when tuning hyperparameters.
3. Overfitting to validation sets: Too many iterations can cause the model to overfit to the validation sets.

Small train dataset considerations:

1. Data augmentation: Consider augmenting the train dataset to increase its size.
2. Transfer learning: Leverage pre-trained models or transfer learning to utilize knowledge from larger datasets.
3. Regularization techniques: Implement regularization (e.g., L1, L2, dropout) to prevent overfitting.

Small validation dataset considerations:

1. Increase k cautiously: Monitor convergence and stop increasing k when the estimate stabilizes.
2. Stratified k-fold cross-validation: Use stratified k-fold cross-validation for classification problems to maintain class balance.
3. Alternative validation methods: Consider using techniques like Monte Carlo cross-validation or bootstrap sampling.

Minimum dataset size guidelines:

1. Train dataset: Aim for at least 10-20 samples per feature or 100-200 total samples.
2. Validation dataset: Aim for at least 5-10 samples per class or 20-50 total samples.

Alternatives to k-fold cross-validation:

1. Leave-one-out cross-validation (LOOCV): Suitable for very small datasets, but computationally expensive.
2. Bootstrap sampling: Resample the dataset with replacement to estimate test accuracy.
3. Monte Carlo cross-validation: Randomly split the dataset into train and validation sets.

While increasing iterations can help with small datasets, it's essential to consider these limitations and alternative approaches.

Would you like me to elaborate on any of these points or provide examples?